In [176]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os
from time import sleep, time
from datetime import date, timedelta

In [177]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [178]:
base_url = 'https://spotifycharts.com/regional/us/weekly/latest'
driver = webdriver.Chrome(chromedriver)
driver.get(base_url)

In [179]:
#function for going through each row in each url and finding relevant song info

def song_scrape(songs):
    for tr in songs.find("tbody").findAll("tr"):
        artist = tr.find("td", {"class": "chart-table-track"}).find("span").text
        artist = artist.replace("by ","").strip()
  
        title = tr.find("td", {"class": "chart-table-track"}).find("strong").text
        
        streams = tr.find("td", {"class": "chart-table-streams"}).text
        songid = tr.find("td", {"class": "chart-table-image"}).find("a").get("href")
        songid = songid.split("track/")[1]
    
        url_date= f"{prev_start} - {prev_end}"
        
        final_list.append([title, artist, songid, streams, url_date])

In [181]:
from selenium.common.exceptions import NoSuchElementException

first_date = date(2021, 9, 30)
last_date = date(2020, 9, 30)
diff_end = -6
diff_start = -13
prev_end = (first_date + timedelta(days=-6)).strftime('%Y-%m-%d')
prev_start = (first_date + timedelta(days=-13)).strftime('%Y-%m-%d')
final_list = []

while pd.to_datetime(prev_start).date() > last_date:   
    try:
        filter_button = driver.find_element_by_xpath(
            '//a[contains(@class, "menu-button")]'
        )
        filter_button.click()
        
        date_button = driver.find_element_by_xpath(
            '//div[@data-type="date"]'
        )
        date_button.click()
        
        date_select = driver.find_element_by_xpath(
            f'//li[@data-value="{prev_start}--{prev_end}"]'
        )

        date_select.click()
        
        sleep(1)
        
        html_source = driver.page_source 
        
        soup = BeautifulSoup(html_source, "html.parser")
        songs = soup.find("table", {"class":"chart-table"})
        song_scrape(songs)
        
        sleep(2)
        
    except NoSuchElementException:
        print("No such element. Stopping scrape.")

    prev_end = (pd.to_datetime(prev_end).date() - timedelta(days=7)).strftime('%Y-%m-%d')
    prev_start = (pd.to_datetime(prev_start).date() - timedelta(days=7)).strftime('%Y-%m-%d')
    #print(prev_start, prev_end)


In [184]:
final_df = pd.DataFrame(final_list, columns= ["Title", "Artist", "Song ID", "Streams", "Weekly Chart Date"])
final_df.head()

,Title,Artist,Song ID,Streams,Weekly Chart Date
0,INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,5Z9KJZvQzH6PFmb8SNkxuk,"13,890,659",2021-09-17 - 2021-09-24
1,Way 2 Sexy (with Future & Young Thug),Drake,0k1WUmIRnG3xU6fvvDVfRG,"10,897,624",2021-09-17 - 2021-09-24
2,STAY (with Justin Bieber),The Kid LAROI,5PjdY0CKGZdEuoNab3yDmX,"10,613,177",2021-09-17 - 2021-09-24
3,THATS WHAT I WANT,Lil Nas X,0e8nrvls4Qqv5Rfa2UhqmO,"10,150,153",2021-09-17 - 2021-09-24
4,Knife Talk (with 21 Savage ft. Project Pat),Drake,2BcMwX1MPV6ZHP4tUT9uq6,"9,492,188",2021-09-17 - 2021-09-24


In [183]:
with open('spotifydata_{}_{}.csv'.format(last_date, first_date), 'w') as f:
        final_df.to_csv(f, header= True, index=False)